<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

<h1 align=center><font size = 5>Assignment: SQL Notebook for Peer Assignment</font></h1>

Date completed: 5 October 2024

## Objectives
1.  Understand the Spacex DataSet
2.  Load the dataset  into the corresponding table in a Db2 database
3.  Execute SQL queries to answer assignment questions


## Overview of the DataSet

SpaceX has gained worldwide attention for a series of historic milestones.

It is the only private company ever to return a spacecraft from low-earth orbit, which it first accomplished in December 2010.
SpaceX advertises Falcon 9 rocket launches on its website with a cost of 62 million dollars wheras other providers cost upward of 165 million dollars each, much of the savings is because Space X can reuse the first stage.


Therefore if we can determine if the first stage will land, we can determine the cost of a launch.

This information can be used if an alternate company wants to bid against SpaceX for a rocket launch.

This dataset includes a record for each payload carried during a SpaceX mission into outer space.


## Import Libraries

In [54]:
# Install required libraries
!pip install ipython-sql
import pandas as pd
import sqlite3
from sqlite3 import Error

In [55]:
# Load the SQL extension
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


## Read in CSV file


In [56]:
df = pd.read_csv('/content/Spacex.csv')

# Optional: Display the first few rows to verify
print(df.head())

         Date Time (UTC) Booster_Version  Launch_Site  \
0  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS__KG_  \
0               Dragon Spacecraft Qualification Unit                  0   
1  Dragon demo flight C1, two CubeSats, barrel of...                  0   
2                              Dragon demo flight C2                525   
3                                       SpaceX CRS-1                500   
4                                       SpaceX CRS-2                677   

       Orbit         Customer Mission_Outcome      Landing_Outcome  
0        LEO           SpaceX         Success  Failure (parachute)  
1  LEO (ISS)  NASA (COTS) NRO         Success  Failure (parachute)  

In [57]:
# Clean column names (remove spaces, special characters)
df.columns = [col.replace(' ', '_').replace('(', '').replace(')', '').replace('%', 'pct') for col in df.columns]

### Create a SQLite Database Connection




In [58]:
def create_connection(db_file):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f"Connected to SQLite version: {sqlite3.version}")
        return conn
    except Error as e:
        print(e)

    return conn

# Create a database file (will be created if it doesn't exist)
conn = sqlite3.connect('spacex_data.db')
print("Database connection created successfully")

Database connection created successfully


### Create a Table and Insert Data

In [59]:
# Use pandas to directly write the dataframe to SQLite
if conn is not None:
    # The to_sql function creates a table and populates it
    df.to_sql('SPACEXTABLE', conn, if_exists='replace', index=False)
    print("Data successfully loaded to SQLite database")
else:
    print("Error! Cannot create the database connection.")

Data successfully loaded to SQLite database


In [72]:
# Test query
# Load the ipython-sql extension to enable SQL magic commands
%load_ext sql

# Connect to the database
%sql sqlite:///spacex_data.db

%config SqlMagic.style = 'DEFAULT'

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [76]:
conn = sqlite3.connect('spacex_data.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('SPACEXTBL',), ('SPACEXTABLE',)]


## Tasks

Now write and execute SQL queries to solve the assignment tasks.

**Note: If the column names are in mixed case enclose it in double quotes
   For Example "Landing_Outcome"**

### Task 1




##### Display the names of the unique launch sites  in the space mission


In [84]:
# Task 1: Display the names of the unique launch sites in the space mission
query = 'SELECT DISTINCT "LAUNCH_SITE" FROM SPACEXTABLE;'
launch = pd.read_sql_query(query, conn)
print(launch)

    Launch_Site
0   CCAFS LC-40
1   VAFB SLC-4E
2    KSC LC-39A
3  CCAFS SLC-40



### Task 2


#####  Display 5 records where launch sites begin with the string 'CCA'


In [85]:
# Display 5 records where launch sites begin with the string 'CCA'

query = 'SELECT * FROM SPACEXTABLE WHERE Launch_Site LIKE "CCA%" LIMIT 5;'
cca = pd.read_sql_query(query, conn)
print(cca)

         Date  Time_UTC Booster_Version  Launch_Site  \
0  2010-06-04  18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1  2010-12-08  15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2  2012-05-22   7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3  2012-10-08   0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4  2013-03-01  15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  PAYLOAD_MASS__KG_  \
0               Dragon Spacecraft Qualification Unit                  0   
1  Dragon demo flight C1, two CubeSats, barrel of...                  0   
2                              Dragon demo flight C2                525   
3                                       SpaceX CRS-1                500   
4                                       SpaceX CRS-2                677   

       Orbit         Customer Mission_Outcome      Landing_Outcome  
0        LEO           SpaceX         Success  Failure (parachute)  
1  LEO (ISS)  NASA (COTS) NRO         Success  Failure (parachute)  
2  LE

### Task 3




##### Display the total payload mass carried by boosters launched by NASA (CRS)


In [87]:
# Display the total payload mass carried by boosters launched by NASA (CRS)

query = 'SELECT SUM(PAYLOAD_MASS__KG_) AS "Total Payload Mass (kg)" FROM SPACEXTABLE WHERE Customer LIKE "NASA (CRS)";'
totalmass = pd.read_sql_query(query, conn)
print(totalmass)

   Total Payload Mass (kg)
0                    45596


### Task 4




##### Display average payload mass carried by booster version F9 v1.1


In [89]:
# Display average payload mass carried by booster version F9 v1.1

query = 'SELECT AVG(PAYLOAD_MASS__KG_) AS "Average Payload Mass (kg)" FROM SPACEXTABLE WHERE Booster_Version LIKE "F9 v1.1";'
avgmass = pd.read_sql_query(query, conn)
print(avgmass)

   Average Payload Mass (kg)
0                     2928.4


### Task 5

##### List the date when the first succesful landing outcome in ground pad was acheived.



In [93]:
# List the date when the first succesful landing outcome in ground pad was acheived.

query = 'SELECT MIN(Date) AS "First Successful Landing" FROM SPACEXTABLE WHERE "Landing_Outcome" = "Success (ground pad)";'
firstsuccess = pd.read_sql_query(query, conn)
print(firstsuccess)

  First Successful Landing
0               2015-12-22


### Task 6

##### List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000


In [94]:
# List the names of the boosters which have success in drone ship and have payload mass greater than 4000 but less than 6000

query = 'SELECT Booster_Version FROM SPACEXTABLE WHERE "Landing_Outcome" = "Success (drone ship)" AND PAYLOAD_MASS__KG_ > 4000 AND PAYLOAD_MASS__KG_ < 6000;'
booster = pd.read_sql_query(query, conn)
print(booster)

  Booster_Version
0     F9 FT B1022
1     F9 FT B1026
2  F9 FT  B1021.2
3  F9 FT  B1031.2


### Task 7




##### List the total number of successful and failure mission outcomes


In [100]:
# List the total number of successful and failure mission outcomes

query = 'SELECT Mission_Outcome, COUNT(*) FROM SPACEXTABLE GROUP BY Mission_Outcome;'
mission = pd.read_sql_query(query, conn)
print(mission)

                    Mission_Outcome  COUNT(*)
0               Failure (in flight)         1
1                           Success        98
2                          Success          1
3  Success (payload status unclear)         1


### Task 8



##### List the   names of the booster_versions which have carried the maximum payload mass. Use a subquery


In [101]:
# List the names of the booster_versions which have carried the maximum payload mass. Use a subquery

query = 'SELECT Booster_Version FROM SPACEXTABLE WHERE PAYLOAD_MASS__KG_ = (SELECT MAX(PAYLOAD_MASS__KG_) FROM SPACEXTABLE);'
boosterv = pd.read_sql_query(query, conn)
print(boosterv)

   Booster_Version
0    F9 B5 B1048.4
1    F9 B5 B1049.4
2    F9 B5 B1051.3
3    F9 B5 B1056.4
4    F9 B5 B1048.5
5    F9 B5 B1051.4
6    F9 B5 B1049.5
7   F9 B5 B1060.2 
8   F9 B5 B1058.3 
9    F9 B5 B1051.6
10   F9 B5 B1060.3
11  F9 B5 B1049.7 


### Task 9


##### List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

**Note: SQLLite does not support monthnames. So you need to use  substr(Date, 6,2) as month to get the months and substr(Date,0,5)='2015' for year.**


In [104]:
# List the records which will display the month names, failure landing_outcomes in drone ship ,booster versions, launch_site for the months in year 2015.

query = 'SELECT STRFTIME("%m", Date) AS Month, Landing_Outcome, Booster_Version, Launch_Site \
         FROM SPACEXTABLE WHERE STRFTIME("%Y", Date) = "2015" AND Landing_Outcome = \'Failure (drone ship)\';'
records = pd.read_sql_query(query, conn)
print(records)

  Month       Landing_Outcome Booster_Version  Launch_Site
0    01  Failure (drone ship)   F9 v1.1 B1012  CCAFS LC-40
1    04  Failure (drone ship)   F9 v1.1 B1015  CCAFS LC-40


### Task 10




##### Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.


In [105]:
# Rank the count of landing outcomes (such as Failure (drone ship) or Success (ground pad)) between the date 2010-06-04 and 2017-03-20, in descending order.

query = 'SELECT Landing_Outcome, COUNT(*) AS OutcomeCount FROM SPACEXTABLE WHERE Date BETWEEN "2010-06-04" AND "2017-03-20" GROUP BY Landing_Outcome ORDER BY OutcomeCount DESC;'
rank = pd.read_sql_query(query, conn)
print(rank)

          Landing_Outcome  OutcomeCount
0              No attempt            10
1    Success (drone ship)             5
2    Failure (drone ship)             5
3    Success (ground pad)             3
4      Controlled (ocean)             3
5    Uncontrolled (ocean)             2
6     Failure (parachute)             2
7  Precluded (drone ship)             1


### Reference Links

* <a href ="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20String%20Patterns%20-%20Sorting%20-%20Grouping/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : String Patterns, Sorting and Grouping</a>  

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Built-in%20functions%20/Hands-on_Lab__Built-in_Functions.md.html?origin=www.coursera.org">Hands-on Lab: Built-in functions</a>

*  <a  href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Labs_Coursera_V5/labs/Lab%20-%20Sub-queries%20and%20Nested%20SELECTs%20/instructional-labs.md.html?origin=www.coursera.org">Hands-on Lab : Sub-queries and Nested SELECT Statements</a>

*   <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-3-SQLmagic.ipynb">Hands-on Tutorial: Accessing Databases with SQL magic</a>

*  <a href= "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/Module%205/DB0201EN-Week3-1-4-Analyzing.ipynb">Hands-on Lab: Analyzing a real World Data Set</a>




## Author(s)

<h4> Lakshmi Holla </h4>


## Other Contributors

<h4> Rav Ahuja </h4>


<!--
## Change log
| Date | Version | Changed by | Change Description |
|------|--------|--------|---------|
| 2024-07-10 | 1.1 |Anita Verma | Changed Version|
| 2021-07-09 | 0.2 |Lakshmi Holla | Changes made in magic sql|
| 2021-05-20 | 0.1 |Lakshmi Holla | Created Initial Version |
-->


## <h3 align="center"> © IBM Corporation 2021. All rights reserved. <h3/>
